In [ ]:
import pandas as pd

In [ ]:
!head ../rawdata/monday.csv


In [ ]:
dfs=[]
shiftid=0
for day in ["monday", "tuesday", "wednesday", "thursday", "friday"]:
    df=pd.read_csv("../rawdata/"+day+".csv", delimiter= ";", infer_datetime_format=True)
    df["timestamp"]=df["timestamp"].apply(lambda x :  pd.Timestamp(x))
    df=df.set_index("timestamp")
    df["cID"] = (df["customer_no"]+shiftid).astype("str")
    #df["cID"] = df["customer_no"].apply(lambda x: day+"_"+str(x))
    df=df.drop("customer_no", axis=1)
    shiftid=len(df)
    dfs.append(df)

for i,df in enumerate(dfs):
    if i==0: 
        df_supermarket=df
    if i>0:
        df_supermarket= pd.concat([df_supermarket,df], axis=0)
    

In [ ]:
df_supermarket=df_supermarket.groupby("cID").resample("60s").first().ffill()

In [ ]:
df_supermarket=df_supermarket.drop("cID", axis=1).reset_index("cID")

In [ ]:
entry_df=pd.DataFrame(df_supermarket.reset_index("timestamp").groupby("cID").first()["timestamp"]-pd.Timedelta(60*1e9))

In [ ]:
entry_df["location"] = len(entry_df)* ["entry"]

In [ ]:
df_supermarket.reset_index(inplace=True)
entry_df.reset_index(inplace=True)

In [ ]:
entry_df=entry_df[df_supermarket.columns]

In [ ]:
df_supermarket=pd.concat([entry_df, df_supermarket], axis=0)

In [ ]:
df_supermarket.reset_index(drop=True, inplace=True)

In [ ]:
df_supermarket[df_supermarket["cID"]=="1"]

In [ ]:
IDs_with_checkout=df_supermarket.groupby("location").get_group("checkout")["cID"]

In [ ]:
allIDs=df_supermarket["cID"].unique()

In [ ]:
IDs_without_checkout=list(set(allIDs) - set(IDs_with_checkout))

In [ ]:
df_subset_no_checkout=df_supermarket[df_supermarket["cID"].isin(IDs_without_checkout)]

In [ ]:
checkout_df=pd.DataFrame(df_subset_no_checkout.groupby("cID").last()["timestamp"]+pd.Timedelta(60*1e9))

In [ ]:
checkout_df["location"] = len(checkout_df)* ["checkout"]

In [ ]:
checkout_df.reset_index(inplace=True)

In [ ]:
checkout_df=checkout_df[df_supermarket.columns]

In [ ]:
df_supermarket=pd.concat([df_supermarket, checkout_df, ], axis=0)

In [ ]:
df_supermarket[df_supermarket["cID"]==IDs_without_checkout[1]]

In [ ]:
df_supermarket.memory_usage()

In [ ]:
df_supermarket

In [ ]:
import pickle
pickle.dump(df_supermarket,open("df_supermarket.pickle", "wb"))